# Resultados gerais

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
from generate_tsp import GerarProblemaRadialTSP
from sga import SGAPermutacao, SGAInteiroLimitado

In [3]:
def EscreverResultado(res,params,topologia,tipo,vetorOtimo):
    params['Topologia'] = topologia
    pop,stats,hof,params,custo = res
    resPerm = pd.Series(params)
    resPerm['Tipo'] = tipo
    minVal, gap, success, optSuccess = stats.select("min",'gap','success','optimal success')
    resPerm['Sucesso fraco'] = np.any(success)
    resPerm['Sucesso'] = np.any(optSuccess)
    resPerm['Geração final fraco'] = np.argwhere(success)[0][0] if np.any(success) else len(success)
    resPerm['Geração final'] = np.argwhere(optSuccess)[0][0] if np.any(optSuccess) else len(optSuccess)
    resPerm['MBF'] = np.mean(minVal)
    resPerm['Valor encontrado'] = hof[0].fitness.values[0]
    resPerm['Gap'] = custo(vetorOtimo)[0]/resPerm['Valor encontrado']
    resPerm['Indivíduo encontrado'] = hof[0]
    return pd.DataFrame([resPerm])
    
def Experimento(posicoes,params,nomePosicao='Padrão',vetorOtimo=None):
    res = SGAPermutacao(posicoes,params,vetorOtimo=vetorOtimo)
    resPerm = EscreverResultado(res,params,nomePosicao,'Permutação',vetorOtimo)
    res = SGAInteiroLimitado(posicoes,params,vetorOtimo=vetorOtimo)
    resInt = EscreverResultado(res,params,nomePosicao,'Inteiro',vetorOtimo)
    return pd.concat([resPerm,resInt],ignore_index=True)

In [23]:
def CalcularSR(resultados):
    return (100*resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso'].sum()/resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso'].count()).unstack(['Topologia',"Tipo"])
CalcularMBF = lambda resultados: resultados.groupby(['Topologia','Tipo','P','mu'])['MBF'].describe().reset_index().set_index(['Topologia','Tipo','P','mu']).unstack(['Topologia','Tipo'])
CalcularAES = lambda resultados: resultados[resultados['Sucesso'] == True].groupby(['Topologia','Tipo','P','mu'])['Geração final'].describe().unstack(['Topologia','Tipo'])
CalcularGap = lambda resultados: resultados.groupby(['Topologia','Tipo','P','mu'])['Gap'].describe().reset_index().set_index(['Topologia','Tipo','P','mu']).unstack(['Topologia','Tipo'])
def CalcularSRFraco(resultados):
    return (100*resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso fraco'].sum()/resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso fraco'].count()).unstack(['Topologia',"Tipo"])
CalcularAESFraco = lambda resultados: resultados[resultados['Sucesso fraco'] == True].groupby(['Topologia','Tipo','P','mu'])['Geração final fraco'].describe().unstack(['Topologia','Tipo'])

In [5]:
dbconn = sqlite3.connect("Resultados-finais.sqlite")
resultados = pd.read_sql("SELECT * FROM resultados",dbconn)
resultados

,index,N,P,mu,cxpb,mupb,muidxpb,Topologia,Tipo,Sucesso fraco,Sucesso,Geração final fraco,Geração final,MBF,Valor encontrado,Gap,Indivíduo encontrado
0,0,1000,11,20,0.7,0.2,0.05,Ordenada,Permutação,1,1,109,109,6.473602,6.180340,1.000000,b'\n\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x...
1,1,1000,11,20,0.7,0.2,0.05,Ordenada,Inteiro,1,1,40,40,6.227524,6.180340,1.000000,b'\x08\x00\x00\x00\t\x00\x00\x00\t\x00\x00\x00...
2,2,1000,11,20,0.7,0.2,0.05,Aleatória,Permutação,1,1,252,252,6.846630,6.180340,1.000000,b'\x08\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x...
3,3,1000,11,20,0.7,0.2,0.05,Aleatória,Inteiro,1,1,341,341,6.784231,6.180340,1.000000,b'\t\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x...
4,0,1000,11,100,0.7,0.2,0.05,Ordenada,Permutação,1,1,11,11,6.206155,6.180340,1.000000,b'\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,3,1000,11,100,0.7,0.2,0.05,Aleatória,Inteiro,0,0,1001,1001,9.042760,7.295413,0.847154,b'\x05\x00\x00\x00\t\x00\x00\x00\x03\x00\x00\x...
3596,0,1000,11,50,0.7,0.2,0.05,Ordenada,Permutação,1,1,39,39,6.281230,6.180340,1.000000,b'\x04\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00...
3597,1,1000,11,50,0.7,0.2,0.05,Ordenada,Inteiro,1,1,12,12,6.208499,6.180340,1.000000,b'\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00...
3598,2,1000,11,50,0.7,0.2,0.05,Aleatória,Permutação,1,1,13,13,6.221852,6.180340,1.000000,b'\x06\x00\x00\x00\x07\x00\x00\x00\x04\x00\x00...


In [27]:
resultados[(resultados['Sucesso'] == False) & (resultados['Gap'] >= 0.9)].groupby(['Topologia','Tipo',"P","mu"])['N'].count().to_latex("tables/Impacto definição fraco.tex")

In [32]:
CalcularSR(resultados)

Topologia Aleatória            Ordenada           
Tipo        Inteiro Permutação  Inteiro Permutação
P  mu                                             
11 20          23.0      100.0    100.0      100.0
   50          66.0      100.0    100.0      100.0
   100         75.0      100.0    100.0      100.0
21 20           0.0       30.0     99.0       22.0
   50           0.0       52.0    100.0       54.0
   100          1.0       59.0    100.0       66.0
31 20           0.0        0.0     95.0        0.0
   50           0.0        7.0    100.0        1.0
   100          0.0       13.0    100.0       17.0

In [29]:
CalcularMBF(resultados)[['mean']].to_latex('tables/MBF médio.tex')
CalcularMBF(resultados)[['std']].to_latex('tables/MBF std.tex')

In [30]:
CalcularAES(resultados)[['mean']].to_latex('tables/AES médio.tex')
CalcularAES(resultados)[['std']].to_latex('tables/AES std.tex')

In [31]:
CalcularGap(resultados)[['mean']].to_latex('tables/Gap médio.tex')
CalcularGap(resultados)[['std']].to_latex('tables/Gap std.tex')